## 1st Step. Create a Notebook for the Assigment

# My Peer-Graded Assigment: Segmenting and Clustering Neighborhoods in Toronto

First Part is to install the proper libraries

In [ ]:
#Installin Beautiful Soup
import sys
!{sys.executable} -m pip install bs4

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import os
import codecs

## 2nd Step. Get the information form Wikipedia

Second Part of the assigment is to create the dataframe in pandas of the neighborhood of Toronto

In [ ]:
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
header = {
    'User-Agent': 'Mozilla/5.0'
} # Needed to prevent 403 error on Wikipedia
page = requests.get(wiki, headers=header)
soup = BeautifulSoup(page.content)

tables = soup.findAll("table", {"class": "wikitable"})

# show tables
for table in tables:
    print("###############")
    print(table.text[:100])

for tn in range(len(tables)):
    table = tables[tn]

    # preinit list of lists
    rows = table.findAll("tr")
    row_lengths = [len(r.findAll(['th', 'td'])) for r in rows]
    ncols = max(row_lengths)
    nrows = len(rows)
    data = []
    for i in range(nrows):
        rowD = []
        for j in range(ncols):
            rowD.append('')
        data.append(rowD)

    # process html
    for i in range(len(rows)):
        row = rows[i]
        rowD = []
        cells = row.findAll(["td", "th"])
        for j in range(len(cells)):
            cell = cells[j]

            #lots of cells span cols and rows so lets deal with that
            cspan = int(cell.get('colspan', 1))
            rspan = int(cell.get('rowspan', 1))
            l = 0
            for k in range(rspan):
                # Shifts to the first empty cell of this row
                while data[i + k][j + l]:
                    l += 1
                for m in range(cspan):
                    data[i + k][j + l + m] += cell.text

        data.append(rowD)

    # write data out to tab seperated format
    page = os.path.split(wiki)[1]
    fname = 'output_{}_t{}.csv'.format(page, tn)
    f = codecs.open(fname, 'w')
    for i in range(nrows):
        rowStr = '\t'.join(data[i])
        rowStr = rowStr.replace('\n', '')
        print(rowStr)
        f.write(rowStr + '\n')
    f.close()

## 3rd Step. Create the information in Pandas Dataframe

First we put the info in a data frame and clean up the data as requested

In [ ]:
df = pd.read_csv('output_List_of_postal_codes_of_Canada:_M_t0.csv', sep='\t')
df.head()

In [ ]:
df.shape

 ### *Ignore rows with Not assigned*

In [ ]:
# Show the first 15 rows of the original dataframe and the shape to compare
print("Given Dataframe :\n", df.head(15))  
print("Shape of Given Dataframe", df.shape)

# Filter rows that does not content "Not assigned"
options = ['Not assigned']
df_tor = df.loc[~df['Borough'].isin(options)] 

# Show the first 15 rows of the resultant dataframe and shape
print('\nResult Dataframe :\n', df_tor.head(15)) 
print("Shape of Result Dataframe", df_tor.shape)

 ### *Combine Rows with same Postcode adding Neighbourhood names with commas*

In [ ]:
#Combine duplicated rows by groupby
df_tor1 = (df_tor.groupby(['Postcode','Borough'])['Neighbourhood']
       .apply(lambda x: ','.join(set(x.dropna())))
       .reset_index())

# Show the first 15 rows of the dataframe and shap
print('Dataframe :\n', df_tor1.head(15)) 
print("Shape of Dataframe", df_tor1.shape)

 ### *Assign Neighborhood name in the Not Assigned Cells*

In [ ]:
df_tor1.loc[df_tor1['Neighbourhood']=="Not assigned",'Neighbourhood']=df_tor1.loc[df_tor1['Neighbourhood']=="Not assigned",'Borough']
print('Dataframe :\n', df_tor1.tail(20)) 
print("Shape of Dataframe", df_tor1.shape)

## Step 4. Put the file in my repostery Github

Ready "https://github.com/SuperIU/Coursera_Capstone/blob/master/Segmenting%20and%20Clustering%20Neighborhoods%20in%20Toronto.ipynb"

## Step 5. Assign coordenates for the postal codes in Toronto

### *First we import the coodinates form the site given in the Assigment*

In [35]:
# Get the file given in the Assigment
!wget -q -O 'Geopastical_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [36]:
# Get a dataframe with the postal codes and see if the list have the same rows of the Toronto DF
dfll = pd.read_csv('Geopastical_Coordinates.csv')
print('Postcode in Toronto :\n',dfll.head())
print("Shape of Postcode", dfll.shape)

Postcode in Toronto :
   Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
Shape of Postcode (103, 3)


### *We check both dataframes have the same rows. Now we orden in ascendent order both df and get the new columns*

In [37]:
# Order both df in ascending order
df_tor1.sort_values('Postcode')
dfll.sort_values('Postal Code')
#Check both dataframes heads and tails
print('First Rows of Toronto Post codes :\n',df_tor1.head())
print('Fists Rows of Coordinates Toronto :\n',dfll.head())
print('Last Rows of Toronto Post codes :\n',df_tor1.tail())
print('Last Rows of Coordinates Toronto :\n',dfll.tail())

First Rows of Toronto Post codes :
   Postcode      Borough                         Neighbourhood
0      M1B  Scarborough                         Malvern,Rouge
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2      M1E  Scarborough       Morningside,Guildwood,West Hill
3      M1G  Scarborough                                Woburn
4      M1H  Scarborough                             Cedarbrae
Fists Rows of Coordinates Toronto :
   Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
Last Rows of Toronto Post codes :
     Postcode    Borough                                      Neighbourhood
98       M9N       York                                             Weston
99       M9P  Etobicoke                                          Westmount
100      M9R  Etobicoke  Richview Gardens,Kingsview Village,St. Phillip...
101 

In [38]:
# Add the latitude and longitude columns in df_tor1 with the values of dfll
df_tor1['Latitude']=dfll['Latitude']
df_tor1['Longitude']=dfll['Longitude']
# Check the new dataframe 
print('First Rows of Toronto Post codes :\n',df_tor1.head())
print("Shape of Dataframe", df_tor1.shape)

First Rows of Toronto Post codes :
   Postcode      Borough                         Neighbourhood   Latitude  \
0      M1B  Scarborough                         Malvern,Rouge  43.806686   
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union  43.784535   
2      M1E  Scarborough       Morningside,Guildwood,West Hill  43.763573   
3      M1G  Scarborough                                Woburn  43.770992   
4      M1H  Scarborough                             Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  
Shape of Dataframe (103, 5)


# Step 6. Create a notebook with the result